In [2]:
from urllib.request import urlopen
from data_generator import DataGenerator
import tensorflow as tf
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator
from data_generator import DataGenerator


In [3]:
K = tf.keras.backend

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall


    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [12]:
restnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
output = restnet.layers[-1].output
output = tf.keras.layers.Flatten()(output)
restnet = tf.keras.Model(inputs=restnet.input, outputs=output)
for layer in restnet.layers:
    layer.trainable = False

model = tf.keras.Sequential()
model.add(restnet)
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(lr=2e-5),
    metrics=['accuracy', f1, 'mse']
)

38977536/94668760 [===========>..................] - ETA: 5:38

KeyboardInterrupt: 

In [8]:
df = pd.read_csv('data/train_dataset.csv')
df_train = pd.DataFrame()
df_val = pd.DataFrame()

for idx, group in df.groupby('class'):
    train = group.sample(frac=0.8)
    val = group.drop(train.index)
    df_train = df_train.append(train, ignore_index=True)
    df_val = df_val.append(val, ignore_index=True)

    

In [9]:
def create_aug_gen(image_generator, generator):

    for in_x, in_y in generator:
        g_x = image_generator.flow(255*in_x, in_y, batch_size=in_x.shape[0])
        x, y = next(g_x)
        yield x/255.0, y


In [10]:
train_dicts = list(df_train.T.to_dict().values())
val_dicts = list(df_val.T.to_dict().values())

image_gen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.01,
    zoom_range=[0.9, 1.25],
    horizontal_flip=True,
    vertical_flip=True,
)

data_gen = DataGenerator(
    train_dicts, 
    image_dir='data/images', 
    image_size=(224, 224, 3), 
    batch_size=32
)

data_gen_train = create_aug_gen(image_gen, data_gen)

data_gen_val = DataGenerator(
    val_dicts, 
    image_dir='data/images', 
    image_size=(224, 224, 3), 
    batch_size=1, 
    shuffle=False
)

In [11]:
model.fit_generator(data_gen_train, epochs=1, validation_data=data_gen_val, steps_per_epoch=len(data_gen))

 4/48 [=>............................] - ETA: 8:27 - loss: 0.8655 - accuracy: 0.6719 - f1: 0.4697 - mse: 0.2493

KeyboardInterrupt: 

In [8]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18430510150452609305
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4833784204361903245
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 18317734630208292473
physical_device_desc: "device: XLA_GPU device"
]
